In [71]:
import numpy as np
import random
from copy import copy
from matplotlib import pyplot as plt

from simulation_helpers import generateAdjacenyMatrix
from optimal_policy import computeTransY

from heuristics import selectRandom
from heuristics import highRisk
from heuristics import highConnection
from heuristics import highBelief
from heuristics import sampleBelief
from heuristics import highBeliefRisk
from heuristics import sample

In [72]:
n = 10
L = 2
p = 0.2
q = 0
h = 10

In [80]:
A = generateAdjacenyMatrix(n, 1)
Q = np.ones(n).astype(np.bool)
print sample(selectRandom, Q, h, p, q, L, A, n)

[2. 3. 5. 4. 3. 3. 3. 3. 3. 3.]


In [74]:
A = generateAdjacenyMatrix(n, 1)
Q = np.ones(n).astype(np.bool)
print sample(highRisk, Q, h, p, q, L, A, n)

[2. 3. 5. 5. 5. 5. 5. 5. 5. 5.]


In [75]:
A = generateAdjacenyMatrix(n, 1)
Q = np.ones(n).astype(np.bool)
print sample(highConnection, Q, h, p, q, L, A, n)

[3. 5. 7. 7. 7. 7. 7. 7. 7. 7.]


In [76]:
A = generateAdjacenyMatrix(n, 1)
Q = np.ones(n).astype(np.bool)
print sample(highBelief, Q, h, p, q, L, A, n)

[2. 4. 6. 7. 6. 6. 6. 6. 6. 6.]


In [77]:
A = generateAdjacenyMatrix(n, 1)
Q = np.ones(n).astype(np.bool)
print sample(sampleBelief, Q, h, p, q, L, A, n)

[3. 3. 5. 6. 6. 6. 6. 6. 6. 6.]


In [78]:
A = generateAdjacenyMatrix(n, 1)
Q = np.ones(n).astype(np.bool)
print sample(highBeliefRisk, Q, h, p, q, L, A, n)

[2. 3. 4. 7. 7. 7. 7. 7. 7. 7.]


In [79]:
# plt.plot(a, label = 'rank belief')
# plt.plot(b, label = 'random select')
# plt.plot(c, label = 'effect of the removed')

# plt.legend(loc = "lower right")